# Basic Chat Completion

In [ ]:
import requests, os, json
from dotenv import load_dotenv
load_dotenv()

EURI_API_KEY = os.getenv("EURI_API_KEY")
def generate_completion():
    url = "https://api.euron.one/api/v1/euri/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {EURI_API_KEY}"
    }
    payload = {
        "messages": [
            {
                "role": "user",
                "content": "Write a poem about artificial intelligence"
            }
        ],
        "model": "gpt-4.1-nano",
        "max_tokens": 1000,
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    data = json.dumps(data, indent=2)
    print(data)

generate_completion()

# Text Embeddings & Similarity

In [ ]:
import requests
import numpy as np

def generate_embeddings(text):
    url = "https://api.euron.one/api/v1/euri/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {EURI_API_KEY}"
    }
    payload = {
        "input": text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding

text = "The weather is sunny today."

embedding = generate_embeddings(text)
print(f"Embedding len: {len(embedding)} Embedding: {embedding}")

# https://huggingface.co/Qwen/Qwen3-Embedding-0.6B

In [ ]:
# Requires transformers>=4.51.0
# Requires sentence-transformers>=2.7.0

from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

# We recommend enabling flash_attention_2 for better acceleration and memory saving,
# together with setting `padding_side` to "left":
# model = SentenceTransformer(
#     "Qwen/Qwen3-Embedding-0.6B",
#     model_kwargs={"attn_implementation": "flash_attention_2", "device_map": "auto"},
#     tokenizer_kwargs={"padding_side": "left"},
# )

# The queries and documents to embed
queries = [
    "What is the capital of China?",
    "Explain gravity",
]
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun.",
]

# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
query_embeddings = model.encode(queries, prompt_name="query")
document_embeddings = model.encode(documents)
print(f"Query embeddings shape: {query_embeddings.shape}")
print(f"Document embeddings shape: {document_embeddings.shape}")

# Compute the (cosine) similarity between the query and document embeddings
similarity = model.similarity(query_embeddings, document_embeddings)
print(similarity)
# tensor([[0.7646, 0.1414],
#         [0.1355, 0.6000]])


In [ ]:
import math

class InMemoryVectorStore(VectorStore):
    """
    Extremely simplified vector store for demos and tests.
    Stores all vectors in a list and does a linear scan.
    """

    def __init__(self):
        self._docs: List[Document] = []
        self._embeddings: List[List[float]] = []

    def add_documents(self, docs: List[Document], embeddings: List[List[float]]) -> None:
        assert len(docs) == len(embeddings), "Docs and embeddings must align"
        self._docs.extend(docs)
        self._embeddings.extend(embeddings)

    def _cosine_similarity(self, a: List[float], b: List[float]) -> float:
        dot = sum(x * y for x, y in zip(a, b))
        norm_a = math.sqrt(sum(x * x for x in a))
        norm_b = math.sqrt(sum(y * y for y in b))
        if norm_a == 0 or norm_b == 0:
            return 0.0
        return dot / (norm_a * norm_b)

    def search(self, query_embedding: List[float], top_k: int = 5) -> List[Tuple[Document, float]]:
        scores: List[Tuple[Document, float]] = []
        for doc, emb in zip(self._docs, self._embeddings):
            score = self._cosine_similarity(query_embedding, emb)
            scores.append((doc, score))

        # sort by score descending and take top_k
        scores.sort(key=lambda pair: pair[1], reverse=True)
        return scores[:top_k]


In [ ]:
class LoggingProvider(ModelProvider):
    def __init__(self, provider: ModelProvider, logger):
        super().__init__(provider.config)   # keep same config
        self._provider = provider           # wrapped provider (OpenAI, Anthropic, etc.)
        self._logger = logger

    def generate(self, prompt: str, **kwargs) -> str:
        self._logger.info(f"[{self.config.name}] generate() called")
        result = self._provider.generate(prompt, **kwargs)   # delegate to real provider
        self._logger.info(f"[{self.config.name}] generate() finished")
        return result

    def embed(self, texts: list[str]) -> list[list[float]]:
        self._logger.info(f"[{self.config.name}] embed() called with n={len(texts)}")
        vectors = self._provider.embed(texts)
        self._logger.info(f"[{self.config.name}] embed() finished")
        return vectors


# FAISS Vector Database Implementation:

#FAISS (Facebook AI Similarity Search) is a library for efficient similarity search and clustering of dense vectors.